# 01.9 Data Extraction - Link Bout and Self-Report Objects and Events

This notebook demonstrates how to link physical activity bout events to stress self-report objects by finding the closest mood events within a configurable time period that occur AFTER the bout events
It also links self-report events to physical activity bout objects

## Setup

In [1]:
from datetime import timedelta
import sys
sys.path.append('..')
# Import project modules
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.bout_events import BoutEventManager

## Load OCED Data

In [2]:
# Get OCED Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"player_107631_oced_data_time_bouts_notifications_stress_location_linked.json"
data_dict = dataQuery.load_json(oced_data_file)

## Initialize Bout Event Manager

In [3]:
bout_manager = BoutEventManager()

## Link Bout Events to Report Objects

For each physical activity bout event, find the closest mood event within a 2-hour period (configurable) that occurs AFTER the bout event. If a mood event is found, find its related stress self-report object and relate the bout event to this object.

In [4]:
# Link bout events to report objects with default 2-hour time window
extended_data = bout_manager.link_bout_events_to_report_objects(data_dict)

# Check results
linked_bout_events = [
    event for event in extended_data.get('behaviorEvents', [])
    if event.get('behaviorEventType') == 'physical_activity_bout' and any(
        rel['qualifier'] == 'related_to_report' for rel in event.get('relationships', [])
    )
]
print(f"Number of bout events now linked to stress_self_report objects: {len(linked_bout_events)}")


Processing 132 bout START events
Looking for mood events within 3:00:00 AFTER each bout START event


Linking bout events to report objects: 100%|██████████| 132/132 [00:03<00:00, 33.53it/s]


Linking complete:
- Linked 158 bout events (both START and END) to stress_self_report objects
- Time window used: 3:00:00
- Processed 132 START events to find corresponding END events
Number of bout events now linked to stress_self_report objects: 158


## Link Self-Report Events to Bout Objects

Now we'll add the reverse relationship: for each self-report event, find its related self-report object, check if that object is related to any physical activity event, and if so, link the self-report event to the bout object.

In [5]:
# Link self-report events to bout objects (reverse relationship)
extended_data = bout_manager.link_self_report_events_to_bout_objects(extended_data)

# Check results
linked_self_report_events = [
    event for event in extended_data.get('behaviorEvents', [])
    if event.get('behaviorEventType') == 'mood' and any(
        rel['qualifier'] == 'influenced_by' for rel in event.get('relationships', [])
    )
]
print(f"Number of self-report events now linked to bout objects: {len(linked_self_report_events)}")


Processing 76 mood events
Linking mood events to physical activity bout objects...


Linking mood events to bout objects: 100%|██████████| 76/76 [00:00<00:00, 8373.84it/s]


Linking complete:
- Linked 79 mood events to physical activity bout objects
- Total relationships created: 79
- Relationship qualifier used: 'influenced_by'
- Each mood event can be linked to multiple bout objects
Number of self-report events now linked to bout objects: 28


## Save Extended Data

In [6]:
bout_manager.save_extended_data(
    "player_107631_oced_data_time_bouts_notifications_stress_location_linked_bouts_reports_2.json", 
    extended_data
)


Saving extended data with:
- 132 bout objects
- 264 bout events (132 bouts)
- 688 total objects
- 1176 total behavior events
Saved extended data to: c:\Users\20236075\OneDrive - TU Eindhoven\Documents\GitHub\GameBus-HealthBehaviorMining\notebooks\..\data\transformed\player_107631_oced_data_time_bouts_notifications_stress_location_linked_bouts_reports_2.json
